In [4]:
import mysql.connector
from pypika import Table
from pypika import MySQLQuery as Query
import pandas as pd
import numpy as np

t_clubs = Table('clubs')
t_players = Table('players')
t_formations = Table('formations')
t_teams = Table('teams')
t_games = Table('games')
t_positions = Table('positions')
t_events = Table('events')

In [6]:
import traceback

def get_short(name):
    splitted = name.replace('-',' ').replace(' & ',' ').split(' ')

    try:
        if len(splitted) == 1:
            return splitted[0][:3]
            
        elif len(splitted) == 2:
            return f'{splitted[0][0]}{splitted[1][:2]}'
            
        else:
            return f'{splitted[0][0]}{splitted[1][0]}{splitted[2][0]}'
    except Exception:
        print(f'error for {splitted}\n{traceback.format_exc()}')
        return 'err'

color_df = pd.read_csv('misc/colors/colors_darken.csv')
def get_color(name):
    return color_df.loc[color_df['club'] == name]['darken'].values[0]

def add_clubs(cnx, game_events):
    home_name = game_events['home_team_name'].values[0]
    away_name = game_events['away_team_name'].values[0]
    query = Query.into(t_clubs).columns('ID', 'Name', 'Season', 'Competition', 'Short', 'Color').insert(
                (
                    game_events['home_team_id'].values[0],
                    home_name,
                    game_events['season'].values[0],
                    game_events['competition'].values[0],
                    get_short(home_name),
                    get_color(home_name)
                ), 
                (
                    game_events['away_team_id'].values[0],
                    away_name,
                    game_events['season'].values[0],
                    game_events['competition'].values[0],
                    get_short(away_name),
                    get_color(away_name)
                )
            ).ignore()

    with cnx.cursor() as cursor:
        cursor.execute(query.get_sql(), multi=True)    

def add_players(cnx, game_events):
    player_ids = np.unique(np.concatenate([game_events['playerId'].unique(), game_events['receiverId'].loc[~game_events['receiverId'].isna()].unique()])).astype(int)
    player_names = [np.concatenate([game_events.loc[game_events['receiverId'] == player_id]['receiver'].values, game_events.loc[game_events['playerId'] == player_id]['player'].values])[0] for player_id in player_ids]

    query = Query.into(t_players).columns('ID', 'Name').insert(
                *list(zip(player_ids, player_names))
            ).ignore()

    with cnx.cursor() as cursor:
        cursor.execute(query.get_sql(), multi=True)

def get_possession(events, team_id):
    team_events = events.loc[events['team_id'] == team_id]
    opp_events = events.loc[events['team_id'] != team_id]

    n_team = team_events.shape[0]/team_events['minutes'].values[0]
    n_opp = opp_events.shape[0]/opp_events['minutes'].values[0]
    
    return n_team/(n_team+n_opp)

def add_formations(cnx, game_events):
    game_passes = game_events.loc[(game_events['type'] == 'Pass') & (game_events['outcomeType'] == 'Successful')]
    home_events = game_events.loc[game_events['team_id'] == game_events['home_team_id']]
    home_query = Query.into(t_formations).columns('Vector', 'Label', 'Minutes', 'Possession').insert(
        (
            home_events['formation_vector'].values[0],
            home_events['formation_label'].values[0],
            home_events['minutes'].values[0],
            get_possession(game_passes, game_events['home_team_id'].values[0])
        )
    )
    with cnx.cursor() as cursor:
        cursor.execute(home_query.get_sql(), multi=True)
        cursor.execute('SELECT LAST_INSERT_ID();')
        home_formation_id = cursor.fetchall()[0][0]

    away_events = game_events.loc[game_events['team_id'] == game_events['away_team_id']]
    away_query = Query.into(t_formations).columns('Vector', 'Label', 'Minutes', 'Possession').insert(
        (
            away_events['formation_vector'].values[0],
            away_events['formation_label'].values[0],
            away_events['minutes'].values[0],
            get_possession(game_passes, game_events['away_team_id'].values[0])
        )
    )
    
    with cnx.cursor() as cursor:
        cursor.execute(away_query.get_sql(), multi=True)
        cursor.execute('SELECT LAST_INSERT_ID();')
        away_formation_id = cursor.fetchall()[0][0]

    return {'home': home_formation_id, 'away': away_formation_id}

def add_teams(cnx, game_events, formation_ids):
    home_events = game_events.loc[game_events['team_id'] == game_events['home_team_id']]
    home_query = Query.into(t_teams).columns('ClubId', 'FormationId', 'GameId', 'OppositionId', 'Coach').insert(
        (
            home_events['home_team_id'].values[0],
            formation_ids['home'],
            home_events['game_id'].values[0],
            0,
            home_events['coach'].values[0]
        )
    )
    with cnx.cursor() as cursor:
        cursor.execute(home_query.get_sql(), multi=True)
        cursor.execute('SELECT LAST_INSERT_ID();')
        home_team_id = cursor.fetchall()[0][0]

    away_events = game_events.loc[game_events['team_id'] == game_events['away_team_id']]
    away_query = Query.into(t_teams).columns('ClubId', 'FormationId', 'GameId', 'OppositionId', 'Coach').insert(
        (
            away_events['away_team_id'].values[0],
            formation_ids['away'],
            away_events['game_id'].values[0],
            home_team_id,
            away_events['coach'].values[0]
        )
    )
    with cnx.cursor() as cursor:
        cursor.execute(away_query.get_sql(), multi=True)
        cursor.execute('SELECT LAST_INSERT_ID();')
        away_team_id = cursor.fetchall()[0][0]

    insert_opp_id_query = Query.update(t_teams).set(t_teams.OppositionId, away_team_id).where(t_teams.ID == home_team_id)
    with cnx.cursor() as cursor:
        cursor.execute(insert_opp_id_query.get_sql(), multi=True)

    return {'home': home_team_id, 'away': away_team_id}

def add_game(cnx, game_events, team_ids):
    game_date = game_events['date'].values[0]

    query = Query.into(t_games).columns('ID', 'GameDate', 'Season', 'Competition', 'HomeTeamId', 'AwayTeamId').insert(
                (
                    game_events['game_id'].values[0],
                    game_date,
                    game_events['season'].values[0],
                    game_events['competition'].values[0],
                    team_ids['home'],
                    team_ids['away']
                )
            ).ignore()

    with cnx.cursor() as cursor:
        cursor.execute(query.get_sql(), multi=True)

    return game_events['game_id'].values[0]

def add_positions(cnx, game_events, formation_ids, team_ids):
    game_events.loc[game_events['team_id'] == game_events['home_team_id'], 'formation_id'] = formation_ids['home']
    game_events.loc[game_events['team_id'] == game_events['away_team_id'], 'formation_id'] = formation_ids['away']
    game_events.loc[game_events['team_id'] == game_events['home_team_id'], 'db_team_id'] = team_ids['home']
    game_events.loc[game_events['team_id'] == game_events['away_team_id'], 'db_team_id'] = team_ids['away']

    player_ids = np.unique(np.concatenate([game_events['playerId'].unique(), game_events['receiverId'].loc[~game_events['receiverId'].isna()].unique()])).astype(int)
    player_positions = [np.concatenate([game_events.loc[game_events['receiverId'] == player_id]['receiver_position'].values, game_events.loc[game_events['playerId'] == player_id]['player_position'].values])[0] for player_id in player_ids]
    formations = [np.concatenate([game_events.loc[game_events['receiverId'] == player_id]['formation_id'].values, game_events.loc[game_events['playerId'] == player_id]['formation_id'].values])[0] for player_id in player_ids]
    teams = [np.concatenate([game_events.loc[game_events['receiverId'] == player_id]['db_team_id'].values, game_events.loc[game_events['playerId'] == player_id]['db_team_id'].values])[0] for player_id in player_ids]

    query = Query.into(t_positions).columns('PlayerId', 'FormationId', 'TeamId', 'Position').insert(
                *list(zip(player_ids, formations, teams, player_positions))
            ).ignore()

    with cnx.cursor() as cursor:
        cursor.execute(query.get_sql(), multi=True)

def add_events(cnx, game_events, formation_ids):
    game_events.loc[game_events['team_id'] == game_events['home_team_id'], 'formation_id'] = formation_ids['home']
    game_events.loc[game_events['team_id'] == game_events['away_team_id'], 'formation_id'] = formation_ids['away']

    game_events = game_events[['playerId', 'receiverId', 'player_position', 'receiver_position', 'formation_id', 'time_minutes', 'start_x', 'start_y', 'end_x', 'end_y', 'type', 'outcomeType', 'xt', 'scoring_prob', 'blockedX', 'blockedY', 'goalMouthY', 'goalMouthZ', 'cardType', 'isTouch', 'isGoal', 'isShot', 'isDefAction']]

    query = Query.into(t_events).columns('PlayerId', 'ReceiverId', 'playerPosition', 'receiverPosition', 'FormationId', 'TimeMinutes', 'StartX', 'StartY', 'EndX', 'EndY', 'EventType', 'OutcomeType', 'Xt', 'ScoringProb', 'BlockedX', 'BlockedY', 'GoalMouthY', 'GoalMouthZ', 'CardType', 'IsTouch', 'IsGoal', 'IsShot', 'IsDefAction').insert(
                *list(game_events.itertuples(index=False, name=None))
            ).ignore()

    with cnx.cursor() as cursor:
        cursor.execute(query.get_sql(), multi=True)

In [7]:
cnx = mysql.connector.connect(user='root', password='24082408',
                              host='localhost',
                              database='bsc-explorer')

for filename in os.listdir('input_db/'):
    print(filename)
    input_df = pd.read_csv(f'input_db/{filename}')
    games = input_df['game_id'].unique()

    for game in games:
        game_events = input_df.loc[input_df['game_id'] == game]
        game_events = game_events.where(pd.notnull(game_events), None)

        competition = game_events['competition'].values[0]
        if competition[0].isdigit() or competition in ['Etienne', 'Germain']:
            game_events['competition'] = 'Ligue 1'

        print(f'{game}... ', end='')
        game_events['home_team_id'] = f"{game_events['home_team_id'].values[0]}_{game_events['competition'].values[0][:10]}"
        game_events['away_team_id'] = f"{game_events['away_team_id'].values[0]}_{game_events['competition'].values[0][:10]}"
        game_events['team_id'] = game_events['team_id'].apply(lambda x: f"{x}_{game_events['competition'].values[0][:10]}")

        add_clubs(cnx, game_events)
        add_players(cnx, game_events)
        formation_ids = add_formations(cnx, game_events)
        team_ids = add_teams(cnx, game_events, formation_ids)
        game_id = add_game(cnx, game_events, team_ids)
        add_positions(cnx, game_events, formation_ids, team_ids)
        add_events(cnx, game_events, formation_ids)
        print(f'inserted... ')

cnx.commit()
print(f'commited')

db_input_0 (2).csv
Arsenal_Aston Villa_2016-05-15... inserted... 
Arsenal_Bournemouth_2015-12-28... inserted... 
Arsenal_Crystal Palace_2016-04-17... inserted... 
Arsenal_Everton_2015-10-24... inserted... 
Arsenal_Leicester_2016-02-14... inserted... 
Arsenal_Liverpool_2015-08-24... inserted... 
Arsenal_Man City_2015-12-21... inserted... 
Arsenal_Man Utd_2015-10-04... inserted... 
Arsenal_Newcastle_2016-01-02... inserted... 
Arsenal_Norwich_2016-04-30... inserted... 
Arsenal_Southampton_2016-02-02... inserted... 
Arsenal_Stoke_2015-09-12... inserted... 
Arsenal_Sunderland_2015-12-05... inserted... 
Arsenal_Swansea_2016-03-02... inserted... 
Arsenal_Tottenham_2015-11-08... inserted... 
Arsenal_Watford_2016-04-02... inserted... 
Arsenal_West Brom_2016-04-21... inserted... 
Arsenal_West Ham_2015-08-09... inserted... 
Aston Villa_Arsenal_2015-12-13... inserted... 
Bournemouth_Arsenal_2016-02-07... inserted... 
Crystal Palace_Arsenal_2015-08-16... inserted... 
Everton_Arsenal_2016-03-19... i